# Assignment 02: K-Means Clustering

I want you to implement a function to calculate the k-means clustering of a dataframe of $m$ observations with $n$ attributes.

Your implementation function should have the signature:
    
    cluster_kmeans(df, k)
    
where
    
    df is a Pandas dataframe of m observations with n attributes; m rows with n columns (excluding index;
    the index will contain the label for each observation and is not considered an attribute)
    
    k is the number of clusters to find

The function should return a new dataframe with a single column: the cluster label for each observation.
(Copy the index from the input dataframe into the output dataframe to keep them consistent.).  It should also return the last Sum-of-the-Square-Errors (SSE) from the clustering.

For the proximity measure, you can use Euclidean distance as the metric.  Sum-of-the-Square-Errors is the objective function.  You can assume that the attributes have been standardized (i.e. properly-transformed) prior to k-means being called.

**Keep your final solution notebook tidy.  I will be testing your code by first executing every cell in the notebook and then calling your `cluster_kmeans` with my test data.  If a cell cause the interpreter to throw an exception, I will not be able to test the notebook.**

You may write additional functions and tests in this notebook to write and test your solution. *I should be able to add a cell at the end of this notebook and have it test your function by using  the kernel's "Restart and Run All" feature.*

Start by creating test dataframes where the clusters should be obvious.  Then test the end of the process by being able to plot a dataframe's attributes on a 2D plot with the correct cluster label.  (Since these are test points, you know what that label should be.)  Once you know you can generate test data and plot the solution, begin working on the cluster_kmean's function.  

After playing with a small number of points in test dataframes to check your solution, you may consider using [distribution sampling functions from numpy](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html) to generate points in a cluster for testing.  (Combine the clusters you generate into a single dataframe to test how well your algorithm works.)

Do note that the labels from each call to cluster_kmeans may change: that is cluster 1 and cluster 2 might appear as cluster 2 and cluster 1.  Since you know the original labels from your test data as you generated it yourself, you should be able to figure out which belongs to which original label during testing.

**<span style="color:purple">The project will be due after Spring break on Wednesday, March 11th @ 10PM via Mimir Classroom.</span>**



In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from random import randrange



In [4]:
def generate_2D_blob(X, Y, n_points, label):
    """
    Generate a 2D blob of points.
    
    Args:
        X          tuple (mean, var) for X-dimension
        Y          tuple (mean, var) for Y-dimension
        n_points   number of points
        label      the label to return
        
    Returns:
        DataFrame with columns [x,y,label]
    """
    mean_x, var_x = X
    mean_y, var_y = Y
    x_pts = np.random.normal(mean_x, var_x, n_points)
    y_pts = np.random.normal(mean_y, var_y, n_points)
    df = pd.DataFrame()
    df['x'] = x_pts
    df['y'] = y_pts
    df['label'] = label
    return df

In [5]:
def generate_3D_blob(X, Y, Z, n_points, label):
    """
    Generate a 3D blob of points.
    
    Args:
        X          tuple (mean, var) for X-dimension
        Y          tuple (mean, var) for Y-dimension
        Z          tuple (mean, var) for Y-dimension
        n_points   number of points
        label      the label to return
        
    Returns:
        DataFrame with columns [x,y,z,label]
    """
    mean_x, var_x = X
    mean_y, var_y = Y
    mean_z, var_z = Z
    x_pts = np.random.normal(mean_x, var_x, n_points)
    y_pts = np.random.normal(mean_y, var_y, n_points)
    z_pts = np.random.normal(mean_z, var_z, n_points)
    df = pd.DataFrame()
    df['x'] = x_pts
    df['y'] = y_pts
    df['z'] = z_pts
    df['label'] = label
    return df

In [6]:
def cluster_kmeans(df, k):
    """
    Clusters the m observations of n attributes 
    in the Pandas' dataframe df into k clusters.
    
    Euclidean distance is used as the proximity metric.
    
    Arguments:
        df   pandas dataframe of m rows with n columns (excluding index)
        k    the number of clusters to search for
        
    Returns:
        a m x 1 dataframe of cluster labels for each of the m observations
        retaining the original dataframe's (df's) index
        
        the final Sum-of-Error-Squared (SSE) from the clustering
    """
    dataset = df.astype(float).values.tolist()
    X= df.values #returns a numpy array

In [7]:
def distance(point1, point2):
    dis = point1-point2
    return (np.dot(dis, dis))**.5

In [8]:
def create_centroids(data, num_clusters = 3):
    centroids = []
    a,b = np.shape(data)
    for j in range(0, num_clusters):
        temp = []
        for i in range(0, b):
            maxx = int(max(data.iloc[:,i]))
            maxx
            minx = int(min(data.iloc[:,i]))
            temp.append(randrange(minx, maxx))
        centroids.append(temp)
    return np.asarray(centroids)
#centroids = create_centroids(df[df.columns[:-1]])

In [9]:
def label(data, centroids):
    m, n = np.shape(data)
    lab = []
    for i in range(0, m):
        dis = []
        x, y = np.shape(centroids)
        for j in range(0,x):
            dis.append(distance(centroids[j,:], data.iloc[i, :]))
        lab.append(np.argmin(dis))
    return np.asarray(lab)
#labels = label(df[df.columns[:-1]], centroids)

In [10]:
def graph(data, labels, centroids):
    colors = ['b','g','r','c','m','y','k','w']
    for i in range(0, max(labels)+1):
        plt.scatter(centroids[i,0],centroids[i,1], color = colors[i+2], marker = '<', s = 150)
        for j in range(0, len(labels)):
            if labels[j] == i:
                plt.scatter(data.iloc[j,0], data.iloc[j,1], color = colors[i])
#graph(df[df.columns[:-1]], labels, centroids)                

In [11]:
def update_centroids(data, labels, centroids):
    m,n = np.shape(centroids)
    temp_centroid = np.zeros((m,n))
    count = np.zeros((m,n))
    for j in range(0, len(labels)):
        for k in range(0, n):
            temp_centroid[labels[j],k]+= data.iloc[j,k]
            count[labels[j],k]+= 1
    for i in range(0,m):
        for j in range(0,n):
            if count[i,j] == 0:
                count[i,j] += 1
    return temp_centroid/count

In [12]:
def graph_axis(data, labels, centroids, axis):
    colors = ['b','g','r','c','m','y','k','w']
    for i in range(0, max(labels)+1):
        axis.scatter(centroids[i,0],centroids[i,1], color = colors[i+2], marker = '<', s = 150)
        for j in range(0, len(labels)):
            if labels[j] == i:
                axis.scatter(data.iloc[j,0], data.iloc[j,1], color = colors[i])

In [13]:
def ssefind(df,centroids):
    sum_squares=0
    for ran in range (0,df['label'].max()+1):
        data = df[df.label == ran]
        data=data[data.columns[:-1]]
        m, n = np.shape(data)
        distance_1=0
        for i in range(0, m):
            distance_1+=(distance(centroids[ran,:],data.iloc[i,:])**2)
        sum_squares+=distance_1
    return sum_squares

In [14]:
def diffSSE(SSE):

    if (((SSE[-2]-SSE[-1])/SSE[-2])>0.02):
        return True
    else:
        return False


In [15]:
def cluster_kmeans(df, n_clusters =3, iter = 30):
    SSE=[]
    centroids = create_centroids(df[df.columns[:-1]], num_clusters = n_clusters)
    labels = label(df[df.columns[:-1]], centroids)
    df['label'] = labels
    SSE.append(ssefind(df,centroids))
    centroids = update_centroids(df[df.columns[:-1]], labels, centroids)
    labels = label(df[df.columns[:-1]], centroids)
    df['label'] = labels
    SSE.append(ssefind(df,centroids))
    for i in range(0,iter-3):
        while(diffSSE(SSE)):
            centroids = update_centroids(df[df.columns[:-1]], labels, centroids)
            labels = label(df[df.columns[:-1]], centroids)
            df['label'] = labels
            SSE.append(ssefind(df,centroids))
    #graph(df[df.columns[:-1]], labels, centroids)  
    return df.label.to_frame(), SSE[-1]

In [16]:
pts = generate_2D_blob((10,0.2), (11,0.2), 55, 0)
pts = pts.append(generate_2D_blob((4,0.5), (8,0.3), 45, 1), ignore_index=True)
pts = pts.sample(frac=1)
cluster_kmeans(pts,1)

(    label
 51      0
 44      0
 47      0
 16      0
 55      0
 ..    ...
 26      0
 97      0
 90      0
 84      0
 98      0
 
 [100 rows x 1 columns],
 1136.5281323787003)